In [1]:
import datetime
from functools import partial
import imp
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4, 5, 6, 7'
import json
import h5py
from absl import app, flags, logging
import flax
from flax.traverse_util import flatten_dict
import jax
from jax.sharding import Mesh, NamedSharding, PartitionSpec
from ml_collections import config_flags, ConfigDict
import optax
import tensorflow as tf
import tqdm
import wandb
from octo.data.utils.format import standardize_pytree
import pdb
from octo.model.components.tokenizers import LowdimObsTokenizer
from octo.data.dataset import make_single_dataset
from octo.model.octo_model import OctoModel
from octo.utils.jax_utils import initialize_compilation_cache
from octo.utils.spec import ModuleSpec
from octo.utils.train_callbacks import (
    RolloutVisualizationCallback,
    SaveCallback,
    ValidationCallback,
    VisualizationCallback,
)
from octo.utils.train_utils import (
    check_config_diff,
    create_optimizer,
    format_name_with_config,
    merge_params,
    process_text,
    Timer,
    TrainState,
)
from octo.model.components.action_heads import *
import random
import io
from PIL import Image
from octo.data.utils.format import pytree_display

/tmp/ipykernel_165638/3333269111.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2024-05-16 13:50:25.500675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 13:50:25.500746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 13:50:25.501605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 13:50:26.397271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/data/wulingxuan/miniconda3/lib/python3.9/site-packages/tqdm/aut

In [2]:
text_processor = OctoModel.load_pretrained('/mnt/data_x2/wulingxuan/robot/octo-small/').text_processor

2024-05-16 13:50:31.240879: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:3: failed initializing StreamExecutor for CUDA device ordinal 3: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 25430786048
2024-05-16 13:50:31.240945: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:1: failed initializing StreamExecutor for CUDA device ordinal 1: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 25430786048
2024-05-16 13:50:31.240996: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:2: failed initializing StreamExecutor for CUDA device ordinal 2: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 25430786048
2024-05-16 13:50:31.317639: W external/xla/xla/service/platform_util.c

In [3]:
text_processor.encode('Despite the unpredictable and often chaotic nature of life, individuals who maintain a positive outlook, practice mindfulness, and engage in continuous learning and self-improvement are more likely to navigate challenges successfully, achieve their goals, and lead fulfilling lives filled with meaningful relationships and personal accomplishments.')

{'input_ids': array([[    3,  4868,     8, 24810,    11,   557, 28968,  1405,    13,
           280,     6,  1742,   113,  1961,     3,     1]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
text_processor.encode('Despite the unpredictable and often chaotic nature of life, individuals who maintain a positive outlook, practice mindfulness, and engage in continuous learning and self-improvement are more likely to navigate challenges successfully, achieve their goals.')

{'input_ids': array([[    3,  4868,     8, 24810,    11,   557, 28968,  1405,    13,
           280,     6,  1742,   113,  1961,     3,     1]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}